# Quantum Genetic Algorithm

This notebook shows how to improve the conventional quantum genetic algorithm using nested Grover's search structure to speed up the finding of new better candidate solutions. Or should we say that the `immediate register` is the population we are preserving

### before we start describing our algorithm let's import the necessary stuff needed for running QISKit

In [ ]:
# importing QISKit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, execute, register, get_backend, compile

from qiskit.tools import visualization
from qiskit.tools.visualization import circuit_drawer
import qiskit.extensions.simulator
import matplotlib

import utility as Util
from utility import Grover

# import other necessary stuff
import random
import math
import warnings

import sys, time, getpass
try:
    sys.path.append("../")
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
except:
    print('Qconfig.py not found in parent directory')

### setup for n-bit search space

- n-qubit input register `x` (for selection process)
- n-qubit register `im` (for population-like superposition preserving)
- n-qubit register `st` (for candidate input)
- 1-qubit register for target `out` (or fitness function evaluation)

### Outline of the algorithm

1. randomly initialize `st` 
2. select $j$ from $1 \leq j \leq \sqrt{N}$
3. do grover iteration on `im` and `st` **_(need to revise the wording)_**
4. select $k$ from $1 \leq k \leq \sqrt{N}$
5. do grover iteration on `x` and `im` **_(need to revise the wording)_**
6. observe `x` and set `st = x`
7. repeat step 2 to 6 until `x` is maximized (or some other given condition)

### Explanation on the how this algorithm works

**_#todo list..._**

Let's test out with the simple finding maximum number where our fitness function is just $f(x) = x$ so we don't really need to implement additional unit than just the comparator

In [ ]:
register(qx_config['APItoken'], qx_config['url'])
available_backends()

## let's do some experiments on an interesting question. 

What will happen if we do grover's search (find larget number) if we set the starting value (then fixed value) to some superposition? supppose we know that our `im` will only be in range 12-15. and we do `find_greater` algorithm to this state. Let's not waste time and start our program! 

### First, What will happen if we separate the output of each comparison (`im` and `st`) and (`a` and `im`)

In [ ]:
import pprint

n = 4
N = 2**n

qc = QuantumCircuit()
a = QuantumRegister(n, 'a')
im = QuantumRegister(n, 'im')
st = QuantumRegister(n, 'st')
anc = QuantumRegister(2*n - 1, 'anc')
out_im = QuantumRegister(1, 'out_im')
out_a = QuantumRegister(1, 'out_a')
cs = ClassicalRegister(n, 'cs')

qc.add(a)
qc.add(im)
qc.add(st)
qc.add(anc)
qc.add(out_a)
qc.add(out_im)
qc.add(cs)

# we set st to 12 and use greater_or_equal one time

Util.int_to_qubit(12, qc, st)
Grover.init_grover(qc, a, out_a)
Grover.init_grover(qc, im, out_im)

# do one iteration
Util.greater_equal_comp(qc, im, st, anc, out_im[0])
Grover.inv_around_mean(qc, im, anc)

# do comp greater_equal on a and im
qc_2 = QuantumCircuit()
qc_2.add(a)
qc_2.add(im)
qc_2.add(st)
qc_2.add(anc)
qc_2.add(out_a)
qc_2.add(out_im)
qc_2.add(cs)

Util.greater_equal_comp(qc_2, a, im, anc, out_a[0])
Grover.inv_around_mean(qc_2, a, anc)

# try multiple different iterations
for i in range(1, 10):
    rqc = qc + qc_2
    for _ in range(i - 1):
        rqc = rqc + qc_2
    rqc.measure(a, cs)
    print('do %d iterations' % i)
    device_name = 'local_qasm_simulator'
#     device_name = 'ibmq_qasm_simulator'
    my_backend = get_backend(device_name)
    qobj = compile([rqc], backend=my_backend, shots=100)
    job = my_backend.run(qobj)
    lapse = 0
    interval = 60
    while not job.done:
        print('Status @ {} minutes'.format(lapse))
        print(job.status)
        time.sleep(interval)
        lapse += 1
    print(job.status)
    print(job.id)
    
    result = job.result()
    counts = result.get_counts(rqc)
    counts = {int(k[::-1],2) : v for k,v in counts.items()}
    pprint.pprint(counts, width=1)

In [ ]:
import pprint
from IPython.core.pylabtools import figsize
figsize(20, 10)

n = 4
N = 2**n

qc = QuantumCircuit()
a = QuantumRegister(n, 'a')
st = QuantumRegister(n, 'st')
anc = QuantumRegister(2*n - 1, 'anc')
out_a = QuantumRegister(1, 'out_a')
cs = ClassicalRegister(n, 'cs')

qc.add(a)
qc.add(st)
qc.add(anc)
qc.add(out_a)
qc.add(cs)

# we set st to 12 and use greater_or_equal one time

Util.int_to_qubit(11, qc, st)
Grover.init_grover(qc, a, out_a)

# do one iteration
for i in range(3):
    Util.greater_comp(qc, a, st, anc, out_a[0])
    Grover.inv_around_mean(qc, a, anc)
qc.measure(a, cs)
    
device_name = 'local_qasm_simulator'
my_backend = get_backend(device_name)
qobj = compile([qc], backend=my_backend, shots=100)
job = my_backend.run(qobj)
lapse = 0
interval = 60
while not job.done:
    print('Status @ {} minutes'.format(lapse))
    print(job.status)
    time.sleep(interval)
    lapse += 1
print(job.status)
# print(job.id)

result = job.result()
counts = result.get_counts(qc)
counts = {int(k[::-1],2) : v for k,v in counts.items()}
visualization.plot_histogram(counts)

In [ ]:
n = 4
N = 2**n

qc = QuantumCircuit()
x = QuantumRegister(n, 'a')
im = QuantumRegister(n, 'im')
st = QuantumRegister(n, 'st')
anc = QuantumRegister(2*n-2, 'anc')
out_x = QuantumRegister(1, 'out_x')
out_im = QuantumRegister(1, 'out_im')
cs = ClassicalRegister(n, 'cs')
# rs = ClassicalRegister(n, 'rs')

# empty circuit for easy append
empty_qc = QuantumCircuit()
# the real circuit runs
real_qc = QuantumCircuit()

qc.add(x)
qc.add(im)
qc.add(st)
qc.add(anc)
qc.add(out_x)
qc.add(out_im)
qc.add(cs)
# qc.add(rs)
qc.barrier()

# starting value
cur_best = 0

# Grover.init_grover(qc, x, out_x)
Util.int_to_qubit(cur_best, qc, st)
qc.barrier()
Grover.init_grover(qc, im, out_im)
qc.barrier()

results = list()
results.append(cur_best)

loop = 0
max_loop = 1

while cur_best != 15:
    if loop == max_loop:
        break
    loop += 1
    print('starts at {}'.format(cur_best))
    Grover.init_grover(qc, x, out_x)
    qc.barrier()
    j = random.randrange(1, int(math.sqrt(N + 1)))
    for i in range(j):
        # do grover j times on im and st
        Util.greater_equal_comp(qc, im, st, anc, out_im[0])
        Grover.inv_around_mean(qc, im, anc)
    print('do {} iterations'.format(j))
    
    j = random.randrange(1, int(math.sqrt(N + 1)))
    # do grover j times on im and st
    for i in range(j):
        Util.greater_equal_comp(qc, x, im, anc, out_x[0])
        Grover.inv_around_mean(qc, x, anc)
    print('do {} iterations'.format(j))
    qc.barrier()
    qc.measure(x, cs)
    qc.barrier()
    qc.reset(st)
    qc.cx(x, st)
    qc.reset(x)
    qc.h(x)
    qc.barrier()


# print('here')
# qc.measure(x, cs)
# qc.measure(im, rs)
# Execute circuit
# job = execute([qc], backend='ibmq_qasm_simulator', shots=10)
# job = execute([qc], backend='local_qasm_simulator', shots=10)
# result = job.result()
# print([abs(x)**2 for x in result.get_snapshot('1')])
# print([abs(x)**2 for x in result.get_snapshot('2')])

# counts = result.get_counts(qc)
# print(counts)
# counts = dict((k, v) for (k, v) in counts.items() if v > 10)
# counts = {int(k[::-1],2) : v for k,v in counts.items()}
# print('do grover for {} iteration'.format(j))
# visualization.plot_histogram(counts)    
circuit_drawer(qc)

1. randomly initialize `st` 
2. select $j$ from $1 \leq j \leq \sqrt{N}$
3. do grover iteration on `im` and `st` **_(need to revise the wording)_**
4. select $k$ from $1 \leq k \leq \sqrt{N}$
5. do grover iteration on `x` and `im` **_(need to revise the wording)_**
6. observe `x` and set `st = x`
7. repeat step 2 to 6 until `x` is maximized (or some other given condition)

In [ ]:
qc = QuantumCircuit()
a = QuantumRegister(3, 'a')
b = QuantumRegister(3, 'b')
cs = ClassicalRegister(3, 'cs')
qc.add(a)
qc.add(b)
qc.add(cs)
qc.barrier()

Util.int_to_qubit(3, qc, a)
qc.cx(a, b)
qc.measure(a, cs)
qc.barrier()
qc.cx(a, b)
qc.measure(a, cs)
a = qc.save('1')

job = execute([qc], backend='local_qasm_simulator', shots=10)
result = job.result()
counts = result.get_counts(qc)
print(counts)

circuit_drawer(qc)

In [ ]:
n = 4
N = 2**n

qc = QuantumCircuit()
x = QuantumRegister(n, 'x')
st = QuantumRegister(n, 'st')
anc = QuantumRegister(2*n-2, 'anc')
out_x = QuantumRegister(1, 'out-x')
cs = ClassicalRegister(n, 'cs')
ca = ClassicalRegister

qc.add(x)
qc.add(st)
qc.add(anc)
qc.add(out_x)
qc.add(cs)
qc.barrier()

Grover.init_grover(qc, x, out_x)
qc.h(st)

t = random.randrange(1, int(math.sqrt(N)))
for i in range(0):
    Util.greater_comp(qc, x, st, anc, out_x[0])
    Grover.inv_around_mean(qc, x, anc)

qc.measure(x, cs)
# Execute circuit
job = execute(qc, backend='ibmq_qasm_simulator', shots=1000)
result = job.result()

counts = result.get_counts(qc)
counts = {int(k[::-1],2) : v for k,v in counts.items()}
print('do grover for {} iteration'.format(t))
visualization.plot_histogram(counts)

In [ ]:
# test greater or equal comparator

n = 4
qc = QuantumCircuit()
a = QuantumRegister(n, 'a')
b = QuantumRegister(n, 'b')
anc = QuantumRegister(2*n - 1, 'anc')
target = QuantumRegister(1, 'res')
cs = ClassicalRegister(1, 'cs')
ca = ClassicalRegister(n, 'ca')
cb = ClassicalRegister(n, 'cb')

qc.add(a)
qc.add(b)
qc.add(anc)
qc.add(target)
qc.add(cs)
# qc.add(ca)
# qc.add(cb)

Util.int_to_qubit(6, qc, b)
Util.lesser_equal_comp(qc, a, b, anc, target[0])
qc.measure(target, cs)
# qc.measure(a ,ca)
# qc.measure(b, cb)

for i in range(2**n):
    qc_2 = QuantumCircuit()
    qc_2.add(a)
    Util.int_to_qubit(i, qc_2, a)
    nqc = qc_2 + qc
    # circuit_drawer(nqc)
    job = execute([nqc], backend='local_qasm_simulator', shots=10)
    result = job.result()
    counts = result.get_counts()
    print('compare {} with {}'.format(i, 6))
    print(counts)
    print()
    
# circuit_drawer(qc)